Note: highly draft! Just notes right now.

Some overview text about notebooks -> site, tested. Sphinx. Etc

Things "definitely work" if:

  * you follow all the steps below
  * you are using holoviews for visualization
  * you are using conda, on linux or mac
  
Things might still work if you skip and/or modify steps, or are using other visualization packages, or if you are using windows (but all untested).

Recommend you start from the procedure below, get site building successfully, then begin customizing.

## Prerequisites

1. You want a 'static', fairly self contained site (i.e. just ordinary web server required, you can build and serve locally or copy build dir to server, and things will just work).

2. You have one or more sets of notebooks (e.g. `examples/`, or `examples/getting_started` + `examples/user_guide` + `examples/gallery`). TODO: note about notebook file names when issue decided. And note about titles e.g. no title in first cell, start with l2 headings.

3. The packages you want to use in the notebooks are installed.

3. (optional, for API docs) Any packages for which you want to generate API docs are also installed (which could be via e.g. `pip install -e /path/to/pkg`).

## Installation

0. Ensure you have installed packages necessary to run your notebooks, and any packages for which you wish to generate a reference manual.

1. `conda install -c conda-forge python=3.6* notebook ipython sphinx beautifulsoup4 graphviz`

2. `pip install nbsite`

3. `pip install sphinx_ioam_theme` (or, create/use an alternative theme - currently untested)

## One time setup

You can skip various steps below depending on what you already have. But if you skip a step, you may need to add something to one of your existing files (should be able to figure that out by looking at the corresponding file in `nbsite/tmplate`; e.g. if you have existing conf.py you will need to copy in relevant contents of nbsite/tmplate/conf.py).

1. Make a doc dir: `mkdir doc && cd doc`

2. Create a set of basic pages: `nbsite_from_tmplate.py .` (will not overwrite existing files)

3. Edit conf.py as appropriate for your project.

4. Edit doc/index.rst toctree to match pages in conf.py

5. Edit/delete other pages as desired, or add pages. Be sure to update toctrees in index pages as you do this.

5. Commit initial version of your pages  `git commit -m "something" FAQ.rst Makefile about.rst conf.py index.rst latest_news.html ...` (etc)

## Building site

0. `export MODULE=yourmodule`, `export PROJECT=yourproject` (or just replace as appropriate in commands below)

1. (optional, if you want ref manual) Generate reference manual pages: `nbsite_generate_modules.py ${MODULE} -d ./Reference_Manual -n ${PROJECT} -e tests` (TODO: commit or not...I reckon not)

2. Generate rst containers for notebooks: `nbsite_nbpagebuild.py ${PROJECT} ../examples .` 

3. (optional, if you want gallery) Make gallery: `nbsite_gallery.py ../`

4. Make site: `sphinx-build -b html . ./_build/html`

6. Update cross-notebook links for html: `nbsite_fix_links.py _build/html`

7. Do your notebooks need to access local assets such as images? If so, e.g. `cp -r ../examples/assets _build/html/`.

8. View the result in a browser at localhost:8000: `pushd _build/html && python -m http.server`

## Iterating

### rst pages

* Added, removed, or edited an rst page? Update any toc as necessary (e.g. removing an entry), then ee-run step 4.

### Notebooks

* Added new notebook? Re-run step 2 then proceed as for 'edited', below.

* Removed a notebook? You have to manually delete the corresponding rst container file.

* Edited a notebook? Re-run step 4. It ought to only re-generate what it needs to (i.e. what's changed), but that probably doesn't work quite yet. You may need to manually remove a 'temp evaluated' notebook from _build/html to have a notebook re-run and turned into html. Update: remove the html file too. And if you don't edit the notebook but want it to re-run for another reason e.g. changed deps, then touch the notebook.

### Reference manual

* If you are working on the reference manual


## Customization

### Theme

`html_theme` is `sphinx_ioam_theme`. Set `html_theme_options` in conf.py:

* `logo` and `favicon`: provide paths relative to `html_static_path` (`doc/_static` by default)
* `custom_css`: path relative to `html_static_path` overriding styles. Styles come first from the theme's `main.css` (see [theme issue](https://github.com/ioam/sphinx_ioam_theme/issues/1)), then from nbsite's own css `nbsite/_shared_static/nbsite.css`.
* js TODO

### IPython

conf.py options: timeout, startup code...

## Automatic deployment to gh pages

See nbsite's `.travis.yml` for an example.

0. Remove unnecessary files: `nbsite_cleandisthtml.py ./_build/html take_a_chance`

1. `touch ./_build/html/.nojekyll`

2. Something like this into `.travis.yml` (but modify for your own preferences):

```
deploy:
  provider: pages
  skip_cleanup: true
  github_token: $GITHUB_TOKEN
  local_dir: ./_build/html
  on:
    branch: master
```
